# Fisher Matrix Computation - Case 1 EMRI

This notebook demonstrates computing Fisher information matrices for an EMRI system including:

- 🌟 **EMRI System Setup**: High mass-ratio system with Kerr black hole
- 📡 **LISA Response Integration**: Realistic detector modeling with TDI
- 🧮 **Stable Derivatives**: Robust numerical derivative computation
- 📊 **Fisher Matrix Analysis**: Parameter uncertainty estimation
- 🎯 **Case 1 Parameters**: SNR ~50, prograde orbit configuration

This corresponds to **Case 1** from the validation studies - a standard EMRI with moderate eccentricity and high spin.

## Import Required Packages

Import all necessary packages for EMRI modeling, Fisher matrices, and LISA response.

In [ ]:
# Import relevant EMRI packages
from few.waveform import (
    GenerateEMRIWaveform,
    FastKerrEccentricEquatorialFlux,
)

from few.trajectory.ode import KerrEccEqFlux
from few.trajectory.inspiral import EMRIInspiral
from few.utils.utility import get_p_at_t
from few.utils.geodesic import get_separatrix

# Import StableEMRIFisher components
from stableemrifisher.fisher import StableEMRIFisher

# Standard packages
import numpy as np
import time
import os

# GPU acceleration if available
try:
    import cupy as cp
    xp = cp
    USE_GPU = True
except ImportError:
    xp = np
    USE_GPU = False

# Constants
YRSID_SI = 31558149.763545603  # Seconds in a sidereal year
ONE_HOUR = 60 * 60


ℹ️  Using CPU with NumPy
🚀 All packages imported successfully!
📅 Year in seconds: 3.16e+07


## Define EMRI System Parameters - Case 1

Set up the physical parameters for **Case 1** - a standard EMRI system with:
- High mass ratio (1:10⁵)
- High spin Kerr black hole (a = 0.998)
- Moderate eccentricity (e₀ = 0.73)
- Target SNR ≈ 50

In [ ]:
# ================== CASE 1 PARAMETERS ======================
# EMRI source, eps ~ 1e-5, SNR 50, prograde orbit

# EMRI Case 1 parameters as dictionary
emri_params = {
    # Masses and spin
    "m1": 1e6,        # Primary mass (solar masses)
    "m2": 10,         # Secondary mass (solar masses) 
    "a": 0.998,       # Dimensionless spin parameter (near-extremal)
    
    # Orbital parameters
    "p0": 7.7275,     # Initial semi-latus rectum
    "e0": 0.73,       # Initial eccentricity
    "xI0": 1.0,       # cos(inclination) - equatorial orbit
    
    # Source properties
    "dist": 2.20360838037185,  # Distance (Gpc) - calibrated for target SNR
    
    # Sky location (source frame)
    "qS": 0.8,        # Polar angle
    "phiS": 2.2,      # Azimuthal angle
    
    # Kerr spin orientation
    "qK": 1.6,        # Spin polar angle
    "phiK": 1.2,      # Spin azimuthal angle
    
    # Initial phases
    "Phi_phi0": 2.0,    # Azimuthal phase
    "Phi_theta0": 0.0,  # Polar phase
    "Phi_r0": 3.0,      # Radial phase
    
    # Time domain setup
    "dt": 5.0,        # Time step (seconds)
    "T": 0.01,        # Observation time (years)
}

# Extract parameters to variables for backward compatibility
m1 = emri_params["m1"]
m2 = emri_params["m2"]
a = emri_params["a"]
p0 = emri_params["p0"]
e0 = emri_params["e0"]
xI0 = emri_params["xI0"]
dist = emri_params["dist"]
qS = emri_params["qS"]
phiS = emri_params["phiS"]
qK = emri_params["qK"]
phiK = emri_params["phiK"]
Phi_phi0 = emri_params["Phi_phi0"]
Phi_theta0 = emri_params["Phi_theta0"]
Phi_r0 = emri_params["Phi_r0"]
dt = emri_params["dt"]
T = emri_params["T"]

print("   EMRI Case 1 Configuration:")
print("")
print(f"   Primary mass: {m1:.0e} M☉")
print(f"   Secondary mass: {m2:.0e} M☉")
print(f"   Mass ratio: {m2/m1:.0e}")
print(f"   Spin parameter: {a}")
print(f"   Initial eccentricity: {e0}")
print(f"   Initial semi-latus rectum: {p0}")
print(f"   Distance: {dist:.2f} Gpc")
print(f"   Observation time: {T} years = {T * YRSID_SI:.0f} seconds")

   EMRI Case 1 Configuration:

   Primary mass: 1e+06 M☉
   Secondary mass: 1e+01 M☉
   Mass ratio: 1e-05
   Spin parameter: 0.998
   Initial eccentricity: 0.73
   Initial semi-latus rectum: 7.7275
   Target SNR: 50.0
   Distance: 2.20 Gpc
   Observation time: 0.01 years = 315581 seconds


## Trajectory Analysis

Analyze the EMRI inspiral trajectory to ensure the system is physically reasonable and not plunging.

In [32]:
# Create parameter list for trajectory computation
pars_list = list(emri_params.values()) 

# Set up trajectory module
traj = EMRIInspiral(func=KerrEccEqFlux)

print(" Computing EMRI inspiral trajectory...")

# Compute the full inspiral trajectory
t_traj, p_traj, e_traj, xI_traj, Phi_phi_traj, Phi_r_traj, Phi_theta_traj = traj(
    m1, m2, a, p0, e0, xI0, 
    Phi_phi0=Phi_phi0, Phi_theta0=Phi_theta0, Phi_r0=Phi_r0, 
    T=T
)

print(f" Trajectory Analysis:")
print(f"   Final time: {t_traj[-1] / YRSID_SI:.6f} years")
print(f"   Initial p: {p_traj[0]:.4f}")
print(f"   Final p: {p_traj[-1]:.4f}")
print(f"   Initial e: {e_traj[0]:.4f}")
print(f"   Final e: {e_traj[-1]:.4f}")

 Computing EMRI inspiral trajectory...
 Trajectory Analysis:
   Final time: 0.010000 years
   Initial p: 7.7275
   Final p: 7.7198
   Initial e: 0.7300
   Final e: 0.7290


## Configure LISA Detector Response

Set up realistic LISA detector modeling including orbital motion and Time Delay Interferometry (TDI).

In [33]:
# ========================= RESPONSE FUNCTION SETUP ===============================#

print("  Setting up LISA detector response...")
    
from fastlisaresponse import ResponseWrapper
from lisatools.detector import EqualArmlengthOrbits

# TDI configuration
tdi_kwargs = dict(
    orbits=EqualArmlengthOrbits(use_gpu=USE_GPU),
    order=25,                    # TDI order
    tdi="2nd generation",        # TDI generation 
    tdi_chan="AE",              # Use A and E TDI channels
)

# Parameter indices for sky location
INDEX_LAMBDA = 8  # Ecliptic longitude
INDEX_BETA = 7    # Ecliptic latitude

# Signal processing parameters
t0 = 20000.0  # Discard time at edges (orbital effects)

# ResponseWrapper configuration
ResponseWrapper_kwargs = dict(
    Tobs=T,                      # Observation time
    dt=dt,                       # Time step
    index_lambda=INDEX_LAMBDA,   # Sky position indices
    index_beta=INDEX_BETA,
    t0=t0,                       # Edge time to discard
    flip_hx=True,                # Flip cross polarization
    use_gpu=USE_GPU,             # GPU acceleration
    is_ecliptic_latitude=False,  # Coordinate system
    remove_garbage="zero",       # Edge handling
    **tdi_kwargs
)
    
data_channels = ["TDIA", "TDIE"]
print(f"   TDI channels: {tdi_kwargs['tdi_chan']}")
print(f"   TDI generation: {tdi_kwargs['tdi']}")
print(f"   Data channels: {data_channels}")
    
N_channels = len(data_channels)
print(f"   Number of channels: {N_channels}")
print(f"   GPU acceleration: {USE_GPU}")

  Setting up LISA detector response...
   TDI channels: AE
   TDI generation: 2nd generation
   Data channels: ['TDIA', 'TDIE']
   Number of channels: 2
   GPU acceleration: False


## Configure Waveform Generation

Set up high-precision waveform generation using FastEMRIWaveforms.

In [34]:
# ======================= WAVEFORM CONFIGURATION ==========================

# Waveform class setup
waveform_class = FastKerrEccentricEquatorialFlux
waveform_class_kwargs = dict(
    inspiral_kwargs=dict(
        err=1e-11,  # High precision integration error tolerance
    ),
    sum_kwargs=dict(
        pad_output=True  # Required for plunging waveforms
    ),
    mode_selector_kwargs=dict(
        mode_selection_threshold=1e-5  # Include important modes
    ),
)

# Waveform generator setup
waveform_generator = GenerateEMRIWaveform
waveform_generator_kwargs = dict(
    return_list=False,    # Return single waveform
    frame="detector"      # Generate in detector frame
)

print("  Waveform Generation Configuration:")
print("")
print(f"   Waveform class: {waveform_class.__name__}")
print(f"   Integration error: {waveform_class_kwargs['inspiral_kwargs']['err']}")
print(f"   Mode threshold: {waveform_class_kwargs['mode_selector_kwargs']['mode_selection_threshold']}")
print(f"   Frame: {waveform_generator_kwargs['frame']}")
print(f"   Padding enabled: {waveform_class_kwargs['sum_kwargs']['pad_output']}")

  Waveform Generation Configuration:

   Waveform class: FastKerrEccentricEquatorialFlux
   Integration error: 1e-11
   Mode threshold: 1e-05
   Frame: detector
   Padding enabled: True


## Initialize StableEMRIFisher

Set up the Fisher matrix computation with all configured components.

In [35]:
# ===================== FISHER MATRIX SETUP ========================

# Numerical derivative parameters
der_order = 4        # 4th order finite differences
Ndelta = 8           # Number of step sizes for optimization

print(" Initializing StableEMRIFisher...")
print(f"   Derivative order: {der_order}")
print(f"   Step size optimization points: {Ndelta}")

# Initialize StableEMRIFisher with all components
sef = StableEMRIFisher(
    # Waveform configuration
    waveform_class=waveform_class,
    waveform_class_kwargs=waveform_class_kwargs,
    waveform_generator=waveform_generator,
    waveform_generator_kwargs=waveform_generator_kwargs,
    
    # LISA response
    ResponseWrapper=ResponseWrapper,
    ResponseWrapper_kwargs=ResponseWrapper_kwargs,
    
    # Time domain
    dt=dt,
    T=T,
     
    # Computation settings
    stats_for_nerds=True,        # Additional diagnostic output
    use_gpu=USE_GPU,             # GPU acceleration
    der_order=der_order,         # Derivative order
    Ndelta=Ndelta,              # Step size optimization
    return_derivatives=True,     # Return derivative arrays
    filename=None,               # Don't save to file
    deriv_type="stable",        # Use stable derivative method
)

print(f"✅ StableEMRIFisher initialized successfully!")
print(f"   Ready for Fisher matrix computation with LISA response")
print(f"   Configuration includes realistic noise and detector effects")

 Initializing StableEMRIFisher...
   Derivative order: 4
   Step size optimization points: 8
No noise model provided but response has been provided
Generating and loading default PSD file
No noise model provided but response has been provided
Generating and loading default PSD file

TDI2 A and E with stochastic background.

TDI2 A and E with stochastic background.
Using cpu backend for PSD interpolation
✅ StableEMRIFisher initialized successfully!
   Ready for Fisher matrix computation with LISA response
   Configuration includes realistic noise and detector effects
Using cpu backend for PSD interpolation
✅ StableEMRIFisher initialized successfully!
   Ready for Fisher matrix computation with LISA response
   Configuration includes realistic noise and detector effects


## Define Fisher Matrix Parameters

Select the parameters to include in the Fisher matrix analysis and define step size ranges for stable derivative computation.

In [36]:
# ==================== PARAMETER SELECTION ====================

# Parameters to include in Fisher matrix
param_names = [
    "m1",         # Primary mass
    "m2",         # Secondary mass  
    "a",          # Spin parameter
    "p0",         # Initial semi-latus rectum
    "e0",         # Initial eccentricity
    "dist",       # Distance
    "qS",         # Source polar angle
    "phiS",       # Source azimuthal angle
    "qK",         # Kerr spin polar angle
    "phiK",       # Kerr spin azimuthal angle
    "Phi_phi0",   # Initial azimuthal phase
    "Phi_r0",     # Initial radial phase
]

print(f" Fisher Matrix Parameters ({len(param_names)} total):")
for i, param in enumerate(param_names, 1):
    print(f"   {i:2d}. {param}")

# Define step size ranges for stable derivative computation
# These ranges are optimized based on parameter scales and sensitivity
delta_range = dict(
    m1=np.geomspace(1e-4 * m1, 1e-9 * m1, Ndelta),    # Mass scale
    m2=np.geomspace(1e-2 * m2, 1e-7 * m2, Ndelta),    # Mass scale
    a=np.geomspace(1e-5, 1e-9, Ndelta),               # Dimensionless
    p0=np.geomspace(1e-5, 1e-9, Ndelta),              # Dimensionless
    e0=np.geomspace(1e-5, 1e-9, Ndelta),              # Dimensionless
    dist=np.geomspace(1e-4 * dist, 1e-8 * dist, Ndelta),  # Distance scale
    qS=np.geomspace(1e-3, 1e-7, Ndelta),              # Angular
    phiS=np.geomspace(1e-3, 1e-7, Ndelta),            # Angular
    qK=np.geomspace(1e-3, 1e-7, Ndelta),              # Angular
    phiK=np.geomspace(1e-3, 1e-7, Ndelta),            # Angular
    Phi_phi0=np.geomspace(1e-3, 1e-7, Ndelta),        # Phase
    Phi_r0=np.geomspace(1e-3, 1e-7, Ndelta),          # Phase
)

print(f"\n  Step Size Optimization Ranges:")
for param in param_names:
    range_vals = delta_range[param]
    print(f"   {param:<10}: {range_vals[0]:.1e} → {range_vals[-1]:.1e}")

print(f"\n️  Each parameter will be tested with {Ndelta} different step sizes")
print(f"   to find the numerically optimal finite difference step.")

 Fisher Matrix Parameters (12 total):
    1. m1
    2. m2
    3. a
    4. p0
    5. e0
    6. dist
    7. qS
    8. phiS
    9. qK
   10. phiK
   11. Phi_phi0
   12. Phi_r0

  Step Size Optimization Ranges:
   m1        : 1.0e+02 → 1.0e-03
   m2        : 1.0e-01 → 1.0e-06
   a         : 1.0e-05 → 1.0e-09
   p0        : 1.0e-05 → 1.0e-09
   e0        : 1.0e-05 → 1.0e-09
   dist      : 2.2e-04 → 2.2e-08
   qS        : 1.0e-03 → 1.0e-07
   phiS      : 1.0e-03 → 1.0e-07
   qK        : 1.0e-03 → 1.0e-07
   phiK      : 1.0e-03 → 1.0e-07
   Phi_phi0  : 1.0e-03 → 1.0e-07
   Phi_r0    : 1.0e-03 → 1.0e-07

️  Each parameter will be tested with 8 different step sizes
   to find the numerically optimal finite difference step.


## Compute Fisher Information Matrix

Perform the main Fisher matrix computation with automatic step size optimization.

In [ ]:
# ====================== FISHER MATRIX COMPUTATION ======================

print(" Computing Fisher Information Matrix...")
print("\nThis includes:")
print("   • Computing stable numerical derivatives")
print("   • Optimizing finite difference step sizes")
print("   • Including LISA detector response")
print("WARNING: This computation may take a long time if run on CPUs")
print("   Progress will be shown as parameters are processed.\n")

# Perform Fisher matrix computation

derivs, fisher_matrix = sef(
    emri_params,                    # Unpack parameter list
    param_names=param_names,       # Parameters to analyze
    delta_range=delta_range        # Step size ranges
)

print(f"\n Fisher Matrix Computation Complete!")

🧮 Computing Fisher Information Matrix...

This includes:
   • Generating EMRI waveforms with high precision
   • Computing stable numerical derivatives
   • Optimizing finite difference step sizes
   • Including LISA detector response
   • Realistic noise modeling

⏱️  This computation may take several minutes...
   Progress will be shown as parameters are processed.

Body is not plunging, Fisher should be stable.
wave ndim: 2
Computing SNR for parameters: (1000000.0, 10, 0.998, 7.7275, 0.73, 1.0, 2.20360838037185, 0.8, 2.2, 1.6, 1.2, 2.0, 0.0, 3.0, 5.0, 0.01)
wave ndim: 2
Computing SNR for parameters: (1000000.0, 10, 0.998, 7.7275, 0.73, 1.0, 2.20360838037185, 0.8, 2.2, 1.6, 1.2, 2.0, 0.0, 3.0, 5.0, 0.01)
Waveform Generated. SNR: 0.9206987302835408
The optimal source SNR is <= 20. The Fisher approximation may not be valid!
calculating stable deltas...
Waveform Generated. SNR: 0.9206987302835408
The optimal source SNR is <= 20. The Fisher approximation may not be valid!
calculating sta

KeyboardInterrupt: 

## Extract Parameter Uncertainties

Compute parameter uncertainties from the Fisher matrix covariance and analyze the results.

In [ ]:
# ======================= UNCERTAINTY ANALYSIS =======================

print("📊 Computing Parameter Uncertainties...")

# Compute covariance matrix (inverse of Fisher matrix)
param_cov = np.linalg.inv(fisher_matrix)

# Extract parameter uncertainties and relative errors
print("\n🎯 PARAMETER UNCERTAINTY ANALYSIS")
print("=" * 70)
print(f"{'Parameter':<12} {'Value':<15} {'Uncertainty':<15} {'Relative Error':<12}")
print("-" * 70)

param_values = {
    "m1": m1, "m2": m2, "a": a, "p0": p0, "e0": e0, "dist": dist,
    "qS": qS, "phiS": phiS, "qK": qK, "phiK": phiK, 
    "Phi_phi0": Phi_phi0, "Phi_r0": Phi_r0
}

uncertainties = {}
relative_errors = {}

for k, param in enumerate(param_names):
    uncertainty = param_cov[k, k] ** 0.5
    param_value = param_values[param]
    relative_error = uncertainty / abs(param_value) * 100 if param_value != 0 else np.inf
    
    uncertainties[param] = uncertainty
    relative_errors[param] = relative_error
    
    print(f"{param:<12} {param_value:<15.3e} {uncertainty:<15.3e} {relative_error:<12.3f}%")

print("\n📈 Key Insights:")
print(f"   • Mass parameters typically well-constrained")
print(f"   • Angular parameters may show larger relative errors")
print(f"   • Phase parameters often have moderate constraints")
print(f"   • Results depend on SNR, observation time, and orbit geometry")

## Analysis Summary

Summarize the Fisher matrix analysis results and provide physical interpretation.